In [200]:
#1-1
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_1_1.csv')
#print(df.head())
#print(df.info())

df.loc[:, ['신고일자', '신고시각', '처리일자', '처리시각']] = df.loc[:, ['신고일자', '신고시각', '처리일자', '처리시각']].astype(str)

df['신고일시'] = pd.to_datetime(df['신고일자'] + df['신고시각'], format = '%Y%m%d%H%M%S')
df['처리일시'] = pd.to_datetime(df['처리일자'] + df['처리시각'], format = '%Y%m%d%H%M%S')

(df['처리일시'] - df['신고일시']).dt.total_seconds().head()
df['처리시간'] = (df['처리일시'] - df['신고일시']).dt.total_seconds()

mean_response_time = df.groupby('공장명')['처리시간'].mean().reset_index()
print(mean_response_time.sort_values(by='처리시간').iloc[2,0])


#1-2
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_1_2.csv')
print(df.head())
print(df.info())

df=df.assign(gu = df['STATION_ADDR1'].str.extract(r'([가-힣]+구)'))   #외우자!

cond = ((df['gu'] == '성동구') | (df['gu'] == '마포구'))
result = df.loc[cond, :].groupby('gu')['dist'].mean()
print(result)

#1-3
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_1_3.csv')
print(df.head())
print(df.info())

df['총판매량'] = df[['제품A', '제품B', '제품C', '제품D', '제품E']].sum(axis=1)

df1 = df.copy()

df1[['연도', '월']] = df1['기간'].str.split('_', expand=True)
df1['월'] = df1['월'].str.replace('월', '').astype(int)
df1['분기'] = pd.cut(df1['월'], bins = [0,3,6,9,12], labels = [1,2,3,4], right=True)

quarterly_avg = df1.groupby(['연도', '분기'])['총판매량'].mean().reset_index()
result2 = quarterly_avg.loc[quarterly_avg['총판매량'].idxmax()]
print(result2)

<ipython-input-200-818515688>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['20220722' '20220622' '20220211' '20220120' '20220601' '20220101'
 '20220309' '20220520' '20220525' '20220203' '20220503' '20220924'
 '20220620' '20220918' '20220523' '20220215' '20221018' '20220405'
 '20220124' '20220213' '20220118' '20220520' '20220113' '20220523'
 '20220113' '20220102' '20220118' '20220707' '20221126' '20220717'
 '20220709' '20220331' '20220227' '20220228' '20220502' '20220328'
 '20220208' '20220102' '20220722' '20220429' '20220723' '20220212'
 '20220104' '20220414' '20220607' '20220523' '20220302' '20221008'
 '20220122' '20220124' '20220402' '20220101' '20220207' '20220325'
 '20220217' '20221004' '20220610' '20220427' '20220314' '20220621'
 '20220225' '20220107' '20220418' '20220401' '20220228' '20220429'
 '20220607' '20220921' '20220207' '20220107' '20221010' '20220127'
 '20220131' '20220428' '20220315' '20220115' 

공장D
             rent_date rent_place  using_time  dist bill_type  \
0  2019-03-19 00:42:26     ST-920           6  1660   BIL_002   
1  2019-02-14 10:53:40     ST-148          27  4530   BIL_004   
2  2019-02-22 17:31:49      ST-10          28  4240   BIL_002   
3  2019-01-07 18:39:39     ST-366          41  9930   BIL_005   
4  2019-01-19 16:02:25     ST-248          11  1080   BIL_014   

          STATION_ADDR1 STATION_ID  STATION_LATITUDE  STATION_LONGITUDE  
0   서울특별시 구로구 경인로40길 47     ST-920         37.494995         126.858254  
1      서울특별시 중구 을지로 100     ST-148         37.565990         126.987793  
2      서울특별시 마포구 양화로 93      ST-10         37.552746         126.918617  
3     서울특별시 성동구 한림말길 56     ST-366         37.541363         127.017662  
4  서울특별시 성동구 행당동 347-11     ST-248         37.557350         127.029213  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ----

<ipython-input-200-818515688>:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  quarterly_avg = df1.groupby(['연도', '분기'])['총판매량'].mean().reset_index()


In [201]:
#2
import pandas as pd
import numpy as np
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_2_test.csv')

#2-1 (인포)
#print(train.info())
#print(test.info())
test_id = test['ID']

#2-2 (독-종 분리)
train_X = train.drop(['Target'], axis=1)
train_y = train['Target']
test_X = test.drop(['Target'], axis=1)
test_y = test['Target']

#2-3 (cat-num 분리)
cat_columns = train_X.select_dtypes('object').columns.to_list()
num_columns = train_X.select_dtypes('number').columns.to_list()

#2-4 (검증 데이터 생성)
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.3, random_state=1)

#2-5 (전처리)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
stdscaler = StandardScaler()
oe = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)

train_X_oe = oe.fit_transform(train_X[cat_columns])
valid_X_oe = oe.transform(valid_X[cat_columns])
test_X_oe = oe.transform(test_X[cat_columns])

train_X_std = stdscaler.fit_transform(train_X[num_columns])
valid_X_std = stdscaler.transform(valid_X[num_columns])
test_X_std = stdscaler.transform(test_X[num_columns])

train_X_preprocessed = np.concatenate([train_X_oe, train_X_std], axis=1)
valid_X_preprocessed = np.concatenate([valid_X_oe, valid_X_std], axis=1)
test_X_preprocessed = np.concatenate([test_X_oe, test_X_std], axis=1)

#2-6 (모델적합)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(train_X_preprocessed, train_y)

#2-7 (검증 데이터 -> 성능 파악)
from sklearn.metrics import f1_score
pred_val = rf.predict(valid_X_preprocessed)
print(f1_score(valid_y, pred_val, average='macro'))

#2-8 (최종 예측)
test_pred = rf.predict(test_X_preprocessed)
test_pred = pd.DataFrame(test_pred, columns = ['pred'])

#2-9 (생성, 저장)
result = pd.concat([test_id, test_pred], axis=1)
result.to_csv('result.csv', index=False)
print(pd.read_csv('result.csv').head())

0.5602603402553307
     ID      pred
0  1903  Graduate
1   796  Graduate
2  1060  Graduate
3   791   Dropout
4  3416  Graduate


In [202]:
#3-1
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_3_1.csv')
#print(df.info())
#print(df.head())

#3-1-1
mean_before = round(df['before'].mean(), 2)
std_before = round(df['before'].std(), 2)
mean_after = round(df['after'].mean(), 2)
std_after = round(df['after'].std(), 2)

print(mean_before, std_before, mean_after, std_after)

#3-1-2

from scipy.stats import ttest_rel
statistic, p_value = ttest_rel(df['before'], df['after'])
print(round(statistic,2))

#3-1-3
p_value_sig = p_value.round(3)
if p_value_sig < 0.05:
  result = '기각'
else:
  result = '채택'
print(result)

#3-2
import pandas as pd
import numpy as np
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_3_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_3_2_test.csv')
#print(train.info())
#print(test.info())

#3-2-1
import statsmodels.api as sm
import statsmodels.formula.api as smf

formula = 'purchase ~ age + income + family_members'
model = smf.logit(formula, data=train).fit()
#print(model.summary())

odds = model.params['income']
odds_ratio = np.exp(odds)
print(odds_ratio)

#3-2-2
llf = model.llf
residual_deviance = -2 * llf
print(residual_deviance)

#3-2-3(마지막)

test_X = test[['age', 'income', 'family_members']]
test_y = test['purchase']
test_X = sm.add_constant(test_X)

#model2 = sm.Logit(test_y, test_X).fit()
#print(model2.summary())# -> 새로 적합 하라는 말은 없었음

pred_val = model.predict(test_X)
y_pred_prob = (pred_val >= 0.5).astype(int)

error_rate = np.mean(test_y != y_pred_prob)
print(error_rate)

71.41 11.37 76.3 11.94
-7.9
기각
Optimization terminated successfully.
         Current function value: 0.676174
         Iterations 4
1.0000152084810885
108.18783555703052
0.35
